In [1]:
import numpy as np 
import pandas as pd
from sklearn.naive_bayes import MultinomialNB

import nltk # most comman libraries for Natural Language Process
import re # For regular exp

In [2]:
df = pd.read_csv('gender-classifier-DFE-791531.csv')

In [4]:
df.head(5)

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,���It felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20050 entries, 0 to 20049
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   _unit_id               20050 non-null  int64  
 1   _golden                20050 non-null  bool   
 2   _unit_state            20050 non-null  object 
 3   _trusted_judgments     20050 non-null  int64  
 4   _last_judgment_at      20000 non-null  object 
 5   gender                 19953 non-null  object 
 6   gender:confidence      20024 non-null  float64
 7   profile_yn             20050 non-null  object 
 8   profile_yn:confidence  20050 non-null  float64
 9   created                20050 non-null  object 
 10  description            16306 non-null  object 
 11  fav_number             20050 non-null  int64  
 12  gender_gold            50 non-null     object 
 13  link_color             20050 non-null  object 
 14  name                   20050 non-null  object 
 15  pr

I don't need all these to classify the gender, and hence I will make a new df

In [7]:
df = pd.concat([df.gender, df.description], axis=1)
df.dropna(axis = 0, inplace = True) #dropped the null rows

In [8]:
df.head(5)

,gender,description
0,male,i sing my own rhythm.
1,male,I'm the author of novels filled with family dr...
2,male,louis whining and squealing and all
3,male,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe..."
4,female,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16224 entries, 0 to 20049
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   gender       16224 non-null  object
 1   description  16224 non-null  object
dtypes: object(2)
memory usage: 380.2+ KB


Looks perfect now!

We gonna use nlp techs now

In [12]:
import nltk # natural language took kit
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91829\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\91829\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\91829\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

Male - 0,  Female = 1

In [14]:
df.gender = [1 if gender == "female" else 0 for gender in df.gender]

In [17]:
description_list = [] #Empty list for appending laterwards
for description in df.description:
    description = re.sub("[^a-zA-Z]", " ", description)
    # sub method finds the given pattern ([^a-zA-Z] means, NOT letter like ":") and changes them with " " (space)
    description = description.lower()
    # we need to have all letters lowercase (because A is not equall to a)
    description = nltk.word_tokenize(description)
    # we make a word list from our text
    lemma = nltk.WordNetLemmatizer()
    description = [lemma.lemmatize(word) for word in description]
    # we found the roots of each words with lemma
    description = " ".join(description)
    # after all these steps,we joined the words together and remake our text.
    description_list.append(description)
    # and append these texts into the list we created.

now, we will implement "bag of words" method

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

max_features = 5000 # we will take top 5000 feature 

cv = CountVectorizer(max_features=max_features, stop_words = "english")
# in this method, we remove the stopwords (irrelevant words) in English language. (like "of", "and", "the" etc.)

sparce_matrix = cv.fit_transform(description_list).toarray()

print("top used {} words: {}".format(max_features, cv.get_feature_names()))

top used 5000 words: ['aa', 'aaron', 'abc', 'aberdeen', 'ability', 'able', 'absolute', 'absolutely', 'absurdity', 'abuse', 'ac', 'academia', 'academic', 'academy', 'acc', 'accept', 'accepted', 'accepting', 'access', 'accessible', 'accessory', 'accident', 'accommodation', 'accomplish', 'account', 'accountant', 'accounting', 'ace', 'achieve', 'act', 'acting', 'action', 'active', 'activist', 'activity', 'actor', 'actress', 'actual', 'actually', 'ad', 'adalah', 'adam', 'add', 'added', 'addict', 'addicted', 'addiction', 'addition', 'address', 'admin', 'administrator', 'admirer', 'adopted', 'adoption', 'adorable', 'adore', 'adult', 'advance', 'advanced', 'adventure', 'adventurer', 'advertising', 'advice', 'advise', 'adviser', 'advisor', 'advocacy', 'advocate', 'advocating', 'aerial', 'aerospace', 'aesthetic', 'af', 'afc', 'affair', 'affiliate', 'affiliated', 'affiliation', 'affordable', 'aficionado', 'afraid', 'africa', 'african', 'afrikaner', 'afro', 'afternoon', 'ag', 'age', 'agency', 'age

In [19]:
y = df.iloc[:, 0].values
x = sparce_matrix

1.Gaussian NB: It should be used for features in decimal form. GNB assumes features to follow a normal distribution.

2.MultiNomial NB: It should be used for the features with discrete values like word count 1,2,3...

The value can be the count of each occurrence, so a multinomial distribution can better fit the data, while a Gaussian is slightly more limited by its mean and variance.

In [24]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import accuracy_score

In [31]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

lets run the ml model

In [32]:
nb = MultinomialNB()
nb.fit(x_train, y_train)
# prediction
y_pred = nb.predict(x_test)
accuracy = 100.0 * accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

Accuracy:  67.08782742681048


In [33]:
lr = LogisticRegression(max_iter = 2000)
lr.fit(x_train,y_train)
# prediction
y_pred = lr.predict(x_test)
accuracy = 100.0 * accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

Accuracy:  68.78274268104776
